In [1]:
from torch.utils.data import Dataset
import torch
import numpy as np
import pickle
import zlib
from collections import MutableMapping

class CompressedDict(MutableMapping):

    def __init__(self):
        self.store = dict()

    def __getitem__(self, key):
        return pickle.loads(zlib.decompress(self.store[key]))

    def __setitem__(self, key, value):
        self.store[key] = zlib.compress(pickle.dumps(value))

    def __delitem__(self, key):
        del self.store[key]

    def __iter__(self):
        return iter(self.store)

    def __len__(self):
        return len(self.store)

    def __keytransform__(self, key):
        return key

class PrefDB(Dataset):
    """
    A circular database of preferences about pairs of segments.
    For each preference, we store the preference itself
    (mu in the paper) and the two segments the preference refers to.
    Segments are stored with deduplication - so that if multiple
    preferences refer to the same segment, the segment is only stored once.
    """

    def __init__(self, maxlen):
        self.segments = CompressedDict()
        self.seg_refs = {}
        self.prefs = []
        self.maxlen = maxlen

    def append(self, s1, s2, pref):
        k1 = hash(np.array(s1).tobytes())
        k2 = hash(np.array(s2).tobytes())

        for k, s in zip([k1, k2], [s1, s2]):
            if k not in self.segments.keys():
                self.segments[k] = s
                self.seg_refs[k] = 1
            else:
                self.seg_refs[k] += 1

        tup = (k1, k2, pref)
        self.prefs.append(tup)

        if len(self.prefs) > self.maxlen:
            self.del_first()

    def del_first(self):
        self.del_pref(0)

    def del_pref(self, n):
        if n >= len(self.prefs):
            raise IndexError("Preference {} doesn't exist".format(n))
        k1, k2, _ = self.prefs[n]
        for k in [k1, k2]:
            if self.seg_refs[k] == 1:
                del self.segments[k]
                del self.seg_refs[k]
            else:
                self.seg_refs[k] -= 1
        del self.prefs[n]

    def __len__(self):
        return len(self.prefs)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        s1s = s2s = prefs = []
        
        k1, k2, pref = self.prefs[idx]

        return self.segments[k1], self.segments[k2], pref

/home/kip/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [46]:
db = PrefDB(maxlen=100)
for i in range(100):
    db.append(np.zeros((32, 84, 84, 3)), np.zeros((32, 84, 84, 3)), 0)

In [49]:
val_loader = torch.utils.data.DataLoader(
    db,
    batch_size=8,
    shuffle=False,
    num_workers=8
)

for s1s, s2s, prefs in val_loader:
    #print(np.array(s1s).shape)
    print('b')

b
b
b
b
b
b
b
b
b
b
b
b
b


In [15]:
a = torch.rand((5, 16, 1))
b = a.view(-1,1)

c = b.view(5,16, -1)

print(a[3, 11, 0])
print(c[3, 11, 0])

tensor(0.4465)
tensor(0.4465)


In [55]:
torch.zeros((5, 16, 1)).view(n_,1).shape

In [56]:
reward_ensemble = RewardPredictorEnsemble

TypeError: forward() missing 3 required positional arguments: 'self', 's1s', and 's2s'

In [2]:
from a2c_ppo_acktr.envs import make_vec_envs


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

envs = make_vec_envs('PongNoFrameskip-v4', 42, 4,
                     0.99, '/tmp/gym/', device, False)


Logging to /tmp/openai-2020-11-19-19-59-45-861665
Creating dummy env object to get spaces
ShmemVecEnv worker: got KeyboardInterrupt
ShmemVecEnv worker: got KeyboardInterrupt
ShmemVecEnv worker: got KeyboardInterrupt
ShmemVecEnv worker: got KeyboardInterrupt


In [3]:
rollouts.rewards

(4, 84, 84)

In [7]:
b = envs.observation_space.shape

In [5]:
b = torch.zeros((4,12,84,84))

In [9]:
a = (1, b[1:])
a

(1, (84, 84))